# ProPainter 初始化脚本

这个Notebook将自动完成ProPainter的环境配置和模型下载，适用于Google Colab环境。

## 功能说明
- 🔧 自动安装所需依赖
- 📥 下载预训练模型（约400MB）
- 🧪 验证安装完整性
- 📝 提供使用示例

In [ ]:
# 第一步：检查环境并安装依赖
print("🚀 ProPainter 初始化开始...")

import sys
import subprocess
import os

# 检查GPU
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"✅ GPU: {gpu_name} ({gpu_memory:.1f}GB)")
    else:
        print("⚠️  未检测到GPU，将使用CPU运行")
except:
    print("📦 PyTorch未安装，稍后将自动安装")

# 安装必要的包
required_packages = [
    'torch>=2.0.0',
    'torchvision>=0.15.0', 
    'opencv-python-headless',
    'rapidocr',
    'onnxruntime-gpu',
    'scipy',
    'matplotlib',
    'imageio-ffmpeg',
    'tqdm'
]

print("📦 安装依赖包...")
for package in required_packages:
    print(f"安装: {package}")
    result = subprocess.run([sys.executable, '-m', 'pip', 'install', package], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ {package}")
    else:
        print(f"❌ {package} - 安装失败")
        print(result.stderr)

print("\n✅ 依赖安装完成！")

In [ ]:
# 第二步：下载预训练模型
import os
import requests
from tqdm import tqdm
from pathlib import Path

def download_model(url, filename):
    """下载模型文件"""
    weights_dir = Path('weights')
    weights_dir.mkdir(exist_ok=True)
    
    file_path = weights_dir / filename
    
    if file_path.exists():
        print(f"✅ 已存在: {filename}")
        return True
    
    print(f"📥 下载: {filename}")
    
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        
        with open(file_path, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True
        ) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
        
        print(f"✅ 完成: {filename}")
        return True
        
    except Exception as e:
        print(f"❌ 下载失败 {filename}: {e}")
        if file_path.exists():
            file_path.unlink()
        return False

# 模型下载列表
models = {
    'ProPainter.pth': 'https://github.com/sczhou/ProPainter/releases/download/v0.1.0/ProPainter.pth',
    'recurrent_flow_completion.pth': 'https://github.com/sczhou/ProPainter/releases/download/v0.1.0/recurrent_flow_completion.pth',
    'raft-things.pth': 'https://github.com/sczhou/ProPainter/releases/download/v0.1.0/raft-things.pth',
}

print("🔽 开始下载预训练模型...")
print("⏱️  预计需要3-5分钟（取决于网络速度）")
print("-" * 50)

success_count = 0
for filename, url in models.items():
    if download_model(url, filename):
        success_count += 1

print(f"\n📊 下载结果: {success_count}/{len(models)} 个模型")

if success_count == len(models):
    print("🎉 所有模型下载完成！")
else:
    print("⚠️  部分模型下载失败，请检查网络连接")

In [ ]:
# 第三步：验证安装
def verify_installation():
    """验证ProPainter安装"""
    print("🧪 验证ProPainter安装...")
    print("-" * 40)
    
    # 检查模型文件
    weights_dir = Path('weights')
    required_models = ['ProPainter.pth', 'recurrent_flow_completion.pth', 'raft-things.pth']
    
    model_check = True
    for model in required_models:
        model_path = weights_dir / model
        if model_path.exists():
            size_mb = model_path.stat().st_size / (1024*1024)
            print(f"✅ {model} ({size_mb:.1f}MB)")
        else:
            print(f"❌ {model} - 文件缺失")
            model_check = False
    
    # 检查核心依赖
    try:
        import torch
        import torchvision
        import cv2
        import numpy as np
        from rapidocr import RapidOCR
        print("✅ 核心依赖包: 正常")
        
        # 测试OCR引擎
        ocr = RapidOCR()
        print("✅ OCR引擎: 正常")
        
        # 测试设备
        if torch.cuda.is_available():
            device = torch.device('cuda')
            print(f"✅ CUDA设备: {torch.cuda.get_device_name(0)}")
        else:
            device = torch.device('cpu')
            print("✅ CPU设备: 可用")
        
        dependency_check = True
        
    except Exception as e:
        print(f"❌ 依赖检查失败: {e}")
        dependency_check = False
    
    print("-" * 40)
    
    if model_check and dependency_check:
        print("🎉 验证通过！ProPainter已准备就绪")
        return True
    else:
        print("❌ 验证失败，请重新运行初始化")
        return False

# 执行验证
success = verify_installation()

if success:
    print("\n🎯 下一步: 运行使用示例！")
else:
    print("\n🔄 请重新运行上面的代码块")

In [ ]:
# 第四步：使用示例
print("📝 ProPainter 使用示例")
print("=" * 50)

usage_examples = """
# 1. 基础使用流程

## 步骤1: 生成OCR掩码（去除文字/字幕/水印）
!python generate_ocr_mask.py -i /path/to/your/video.mp4 -o ocr_masks --confidence 0.6

## 步骤2: 使用ProPainter进行视频修复  
!python inference_propainter.py -i /path/to/your/video.mp4 -m ocr_masks/video_name_mask --fp16

# 2. 推荐设置（适用于Colab）

## 内存优化设置
!python inference_propainter.py \\
    -i input_video.mp4 \\
    -m mask_folder \\
    --fp16 \\
    --subvideo_length 40 \\
    --height 720 \\
    --width 1280

# 3. 处理图像序列
!python generate_ocr_mask.py -i /path/to/image/folder -o masks
!python inference_propainter.py -i /path/to/image/folder -m masks/folder_name_mask

# 4. 高级OCR设置
!python generate_ocr_mask.py \\
    -i video.mp4 \\
    -o masks \\
    --confidence 0.7 \\
    --dilation 8 \\
    --margin 15 \\
    --sample_rate 2

# 5. 参数说明

OCR参数:
- --confidence: 文字检测置信度 (0.5-0.8推荐)
- --dilation: 掩码膨胀大小 (5-10推荐)
- --margin: 文字框边距 (10-20推荐)
- --sample_rate: 帧采样率 (1=全部, 2=隔帧)

ProPainter参数:
- --fp16: 使用半精度，节省显存
- --subvideo_length: 子视频长度，控制内存使用
- --width/--height: 处理分辨率
- --save_frames: 保存所有帧图像
"""

print(usage_examples)

# 创建快速测试函数
def create_test_data():
    """创建测试数据"""
    print("\n🧪 创建测试数据...")
    
    # 如果有示例视频，可以在这里添加下载逻辑
    test_script = '''
# 测试脚本示例
import cv2
import numpy as np
from pathlib import Path

# 创建一个简单的测试视频（带文字）
def create_test_video(output_path="test_video.mp4", duration=3, fps=24):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (640, 480))
    
    for i in range(duration * fps):
        # 创建彩色背景
        frame = np.random.randint(100, 200, (480, 640, 3), dtype=np.uint8)
        
        # 添加文字
        cv2.putText(frame, f"Frame {i+1}", (50, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, "Test Watermark", (400, 400), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        out.write(frame)
    
    out.release()
    print(f"✅ 测试视频已创建: {output_path}")

# 运行测试
if __name__ == "__main__":
    create_test_video()
    print("现在可以运行:")
    print("!python generate_ocr_mask.py -i test_video.mp4 -o test_masks")
    print("!python inference_propainter.py -i test_video.mp4 -m test_masks/test_video_mask --fp16")
'''
    
    with open('create_test_video.py', 'w') as f:
        f.write(test_script)
    
    print("✅ 测试脚本已创建: create_test_video.py")
    print("运行 !python create_test_video.py 创建测试视频")

create_test_data()

print("\n🎉 ProPainter 初始化完成！")
print("📚 使用上面的示例代码开始你的视频处理之旅吧！")